In [1]:
%pwd

'c:\\Users\\44787\\Desktop\\projects\\end-to-end-SMS-Spam-classifier\\research'

In [2]:
import os

os.chdir("../")
%pwd

'c:\\Users\\44787\\Desktop\\projects\\end-to-end-SMS-Spam-classifier'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [5]:
from src.SMSClassifier.constants import CONFIG_PATH
from src.SMSClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, config=CONFIG_PATH):
        self.config = read_yaml(config)
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        return data_transformation_config

In [17]:
from src.SMSClassifier.logging import logger
import pandas as pd

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer

from imblearn.over_sampling import RandomOverSampler

In [32]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig) -> None:
        self.config = config

    def prepare_data(self):
        df = pd.read_csv(self.config.data_path, encoding='latin-1')
        # print(df.head(2))

        # remove unnecessary columns
        df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

        # rename columns name
        df = df.rename(columns={'v1': 'target', 'v2': 'msg'})

        # remove duplicates
        df = df.drop_duplicates(keep='first')

        # label encoding
        df.target = df.target.replace({'ham': 0, 'spam': 1})

        # text preprocessing
        df.msg = df.msg.apply(self.clean_msg)
        
        # tfidfvectorizer
        # tfv = TfidfVectorizer(max_features=2500)
        # X = tfv.fit_transform(df.msg).toarray()
        # y = df.target
        
        # # random over sampler
        # smote = RandomOverSampler(random_state=42)
        # X, y = smote.fit_resample(X, y)
        
        
        # # 
        # df = pd.DataFrame(X)
        # df['target'] = y
        
        # # print(df.head(2))
        # # print(df.shape)
        
        # save the dataset
        df.to_csv(os.path.join(self.config.root_dir, 'clean_df.csv.'), index=False)

    def clean_msg(self, msg, stemmer=PorterStemmer(), stop_words=set(stopwords.words('english'))):

        # remove html tags
        soup = BeautifulSoup(msg, 'html.parser')
        clean_text = soup.get_text()

        # convert to lower case and splits up the words
        words = word_tokenize(clean_text.lower())

        filter_words = []

        for word in words:
            # removing the stop words and punctuation
            if word not in stop_words and word.isalpha():
                filter_words.append(stemmer.stem(word))  # words stemming

        return ' '.join(filter_words)

In [33]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.prepare_data()
except Exception as e:
    raise e

[2024-01-30 10:28:39,621: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-30 10:28:39,621: INFO: common: created directory at: artifacts/data_transformation]


C:\Users\44787\AppData\Local\Temp\ipykernel_23676\2279487467.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.target = df.target.replace({'ham': 0, 'spam': 1})
C:\Users\44787\AppData\Local\Temp\ipykernel_23676\2279487467.py:47: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(msg, 'html.parser')
